Approach and Implementation

Step 1: Import state csv into the program by using pandas dataframes
If we are given 325 million synthetic individuals data:
age(19) x gender(2) x ethnicity(6) x income(16) x MSA(2) x state(51) = 996096 total groups

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/stephs/Desktop/AK.csv")
#vm is a global
vm =  pd.read_csv("/Users/stephs/Desktop/variable_mapping.csv")
#we put corresponding age group and other variables based on variable_mapping.csv into the AK.csv
# dem_01 to AGE_GROUP
age_vm = vm[vm["var_id"] == "dem_01"]
age_dict = dict(zip(age_vm["value_id"], age_vm["var_values"]))
df["AGE_GROUP"] = df["dem_01"].map(age_dict)

# dem_02 to GENDER 
gender_vm = vm[vm["var_id"] == "dem_02"]
gender_dict = dict(zip(gender_vm["value_id"], gender_vm["var_values"]))
df["GENDER"] = df["dem_02"].map(gender_dict)

# dem_03 to ETHNICITY
ethnicity_vm = vm[vm["var_id"] == "dem_03"]
ethnicity_dict = dict(zip(ethnicity_vm["value_id"], ethnicity_vm["var_values"]))
df["ETHNICITY"] = df["dem_03"].map(ethnicity_dict)

# dem_13 to INCOME_GROUP 
income_vm = vm[vm["var_id"] == "dem_13"]
income_dict = dict(zip(income_vm["value_id"], income_vm["var_values"]))
df["INCOME_GROUP"] = df["dem_13"].map(income_dict)
#then now we can group those with same traits
grouped = df.groupby(["AGE_GROUP", "GENDER", "ETHNICITY", "INCOME_GROUP", "loc_msa"])

#this part can also make as a function with parameter 'df' from different csv files


Step 2: Query LLM for Voting Probability
You can create a function, so that each time when you want to put something in LLM, you call this function

In [ ]:
def call_LLM(random_sample):
    number_of_prediction = int(input("How many predictions do you want to make? "))
    final_prompt = ''
    
    for _ in range(number_of_prediction):
        first_person = input("What's the first candidate's name? ")
        second_person = input("What's the second candidate's name? ")

        # Assume random_sample is a string like 
        # "22  to 24 years, Population of one race: White, Female in Alaska family income with 
        # Less than $10 000, live in Small town"
        prompt = (
            "If the person's info is: " + random_sample + "\n"
            "What is the probability this person votes for " + first_person + 
            " when choosing between " + first_person + " and " + second_person + "?"
        )
        final_prompt += prompt + '\n\n'

    # Now this prompt is ready to be sent to the LLM
    print("Prompt to send to LLM:\n", final_prompt)
    # Send `final_prompt` to LLM API here
    # return LLM_response



Step 3: randomly select one sample from each group/combination

In [ ]:
import random
# a function that whenever import a new state you can use it.
def process_state(df,state_name, vm):
    for group_key, group_df in grouped:
        # Randomly pick one row from this group (group_df)
        if not group_df.empty:
            random_row = group_df.sample(n=1).iloc[0]
            new_prompt = (
                f"{random_row['AGE_GROUP']}, "
                f"{random_row['ETHNICITY']}, "
                #
                f"{random_row['GENDER']} in {state_name} family income with "
                f"{random_row['INCOME_GROUP']}, live in "
                f"{'Small town' if random_row['loc_msa']== 'S' else 'Large city'}"
            )

    
        # Send this to the LLM prompt function
        call_LLM(new_prompt)


Further enhancement:
now we do 7296 x 51 times calling call_LLM function, if we want to reduce the frequency, we can pass 5 groups at once into LLM